# A deep dive into the surface mass balance computation in COSIPY

In the previous notebooks we've seen how changing different parameters of the model affects the calculation of surface mass balance on our glacier. In order to strengthen our understanding somewhat, in this notebook we will go through how COSIPY is calculating the surface mass balance during a simulation. Lets start from the top.

First we run a simple simulation to have some data to look at.

**The standard imports**

In [ ]:
# Have to change the cwd for the ipython session, otherwise COSIPY
# will look for things in the wrong places.
import os
import sys
# This is not really a good method, if cell is re run we end up in the
# wrong directory.
os.chdir('./../')
sys.path.append(os.getcwd())

In [ ]:
from cosipy.utils import edu_utils
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr

In [ ]:
# Have to tell matplotlib to plot inline
%matplotlib inline

Then we initiate the IO and datasets and run the simulation.

In [ ]:
# IO and datasetes
IO, DATA, RESULTS = edu_utils.create_IO()

In [ ]:
edu_utils.run_model(DATA=DATA, IO=IO, RESULT=RESULTS)

***
## Decomposing the mass balance
The surface mass balance, which is measured in meter water equivalent (m w.e.),  is the result of adding the snowfall ($SF$) and deposition ($D$) together and removing the melt ($M$), the sublimation ($s$) and the evaporation/condensation ($e$)

$$
MB_{surf} = SF + D - M - s - e.
$$

Note that the evaporation or condensation depends on the sign, it can have either a positive or negative impact on the mass balance We can plot these variables from our `RESULTS` dataset

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
mb_vars = ['SNOWFALL', 'surfM', 'SUBLIMATION', 'DEPOSITION', 'EVAPORATION',
           'CONDENSATION']
for var in mb_vars:
    RESULTS[var].plot(ax=ax, label=var)
ax.set_ylabel('m w.e.')    
plt.legend();

For our simulated period the plot shows us that sublimation and snowfall make up the main part of the changing mass balance. But there is also a small contribution from deposition. Others are negligible.

### Snowfall
The **snowfall** can be calculated in two ways. If it is a direct product of the weather station/gcm data, it is only multiplied by a constant (`mult_factor_RRR`) and then used directly in the surface mass balance calculation. 

In the other case, when only the rain rates are provided, **snowfall** is derived using a logistic transfer function

$$
\mathrm{SNOWFALL} = \frac{RRR}{1000.0} \frac{\rho_{ice}}{\rho_{fresh\space snow}} \frac{-tanh((T2 - t_0) - c_1) \cdot c_2) + 1}{2}
$$

where $c_1$ and $c_2$ are the center and spread transfer functions. This smoothly scales the solid precipitation from 100% at $t_0$ and 0% at 2 °C. The density of fresh snow is calculated as a function of the air temperature and the wind velocity.

### Melt

The surface melt is calculated from the energy available for melt ($q_m$) and the latent heat required for melting ice ($L_s$)

$$
\mathrm{M} = \frac{q_m \cdot dt}{L_s \cdot 1000}.
$$

We multiply with dt to get the total energy available during the time step, and divide with $1000$ to convert the result to m w.e.

The energy available for melt is the sum of the radiative fluxes:
- Net short wave radiation
- Net long wave radiation
- Ground heat flux
- Rain heat flux 
- Sensible heat flux
- Latent heat flux

These are available in the `RESULTS` as well so we can take a look at them

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
(RESULTS['LWin'] + RESULTS['LWout']).plot(ax=ax, label='LW net')
# Short wave net is actually not in the Results but we can calculate it
(RESULTS['G'] * (1 - RESULTS['ALBEDO'])).plot(ax=ax, label='SW net')
RESULTS['H'].plot(ax=ax, label='Sensible heat flux')
RESULTS['LE'].plot(ax=ax, label='Latent heat flux')
RESULTS['B'].plot(ax=ax, label='Ground heat flux')
RESULTS['QRR'].plot(ax=ax, label='Rain heat flux')
ax.set_ylabel('Energy [W m$^{-2}$]')
plt.legend();

This plot contains a lot of information but there are some key points
- During the day, the **net short wave** radiation (orange), which act to heat the surface of the glacier is largely compensated by the negative **ground heat flux**. This means that energy is transported into the glacier, slowly heating it.
- During the night, the sign of the **ground heat flux** is reversed when the glacier release energy in the form of outgoing **long wave radiation**.

From the look of it, the net energy of the variables above should be close to zero. Let's take a look

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
RESULTS['ME'].plot(ax=ax);

With this in mind we can dissect the radiative fluxes even further. 

#### Net shortwave radiation and albedo
The **net shortwave radiation** is defined as
$$
q_{sw} = (1-\alpha) \cdot q_G
$$

where $q_G$ is the incoming shortwave radiation and $\alpha$ is the **albedo**. $q_G$ is given by the input data, while the evolution of the albedo is parametrized according to the approach from Oerlemans and Knap ([1998](https://www.cambridge.org/core/journals/journal-of-glaciology/article/1-year-record-of-global-radiation-and-albedo-in-the-ablation-zone-of-morteratschgletscher-switzerland/F30150EB1AD9D62A0C007C22FFAE7B6A)). In it, the snow albedo is a function of the time since the last snowfall ($s$) and an albedo timescale ($\tau^*$) that specifies the speed of the degradation of the albedo from that of fresh snow ($\alpha_s$) to that of firn ($\alpha_f$)

$$
\alpha_{snow} = \alpha_f + (\alpha_s - \alpha_f)\space exp \left(\frac{s}{\tau^*}\right).
$$

As the thickness of the snowpack ($d$) decreases, the surface albedo should approach that of the albedo of ice ($\alpha_i$). This is done by introducing a characteristic snow depth scale ($d^*$) to employ what is known as e-folding. With this, the full surface albedo can be written as 

$$
\alpha = \alpha_s + (\alpha_i - \alpha_s)\space exp \left(\frac{-d}{d^*}\right).
$$

The surface albedo is reset to that of fresh snow when new snow accumulation exceeds a certain threshold.

The fraction of the net shortwave radiation that penetrates the glacier decays with the depth ($z$) from the surface according to

$$
Q(z, t) = \lambda_r q_{sw} \space exp(-z \beta)
$$

where $\lambda_r$ is the fraction of absorbed radiation and $\beta$ the extinction coefficient (both depending on if the integrated depth is snow or ice). 

<div class="alert alert-success">
   Some of the constants in the equations above are possible to configure using the opt_dict. Remember that we changed one of them, the albedo of fresh snow, in the sensitivity study. 
</div>

#### Net longwave radiation

The **net longwave radiation** can be handled in two different ways:
If the input data contains the incoming longwave radiation, the net is calculated as

$$
q_{lw} = q_{lw_{in}} - \epsilon_s \sigma T_0^4,
$$

where $\epsilon_s$ is the surface emissivity, in our case constant $\sim 1$.

If the input data doesn't contain the incoming longwave radiation, this flux  will be parametrized using the Stefan-Boltzman law with the means of the air temperature ($T_{zt}$) and the atmospheric emissivity

$$
\epsilon_a = \epsilon_{cs}(1-N^2) + \epsilon_{cl}N^2,
$$

where $N$ is the cloud cover fraction and $\epsilon_{cs}$ and $\epsilon_{cl}$ are the clear-sky and cloud emissivity. The cloud emissivity is kept constant while the clear-sky emissivity is given by

$$\epsilon_{cs} = 0.23 + 0.433(e_{zt}/T_{zt})^{1/8}.$$

where $e_{zt}$ is the water vapour pressure.

In our case, the input data does not contain any incoming longwave radiation and we thus rely on the cloud cover fraction and the air temperature for the parameterization.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
lns1 = (RESULTS['LWin'] + RESULTS['LWout']).plot(ax=ax, label='LW net', c='C0')
ax2 = ax.twinx()
lns2 = DATA.N.plot(ax=ax2, c='C1', label='N')
ax2.set_title('')
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax.set_ylabel('Energy [W m$^{-2}$]')
ax.legend(lns, labs);

#### Sensible and latent heat flux

The sensible and latent heat flux ($q_{sh}$ & $q_{lh}$) are parametrized using a bulk approach of the flux-gradient similarity. This is done since the model only uses meteorological variables from one height. By using the Stanton number ($C_H$) and Dalton number ($C_E$) the turbulent fluxes can be calculated as
$$
q_{sh} = \rho_a c_p C_H u_{z_v} (T_{z_t} - T_0)
$$
and
$$
q_{lh} = \rho_a L_v C_E u_{z_v} (q_{z_q} - q_0)
$$
with the following constants 

|Symbol | Details|
|:---|:---|
|$\rho_a$|  Air density|
|$c_p$|  Specific heat of air|
|$L_v$|   Latent heat of vaporisation, (replaced by latent<BR> heat of sublimation $L_s$ if $T_0 < T_m$)|
|$u_{z_v}$|  Wind velocity at height $z_t$|
|$T_{z_t}$| Temperature at height $z_t$|
|$q_{z_t}$| Mixing ratio at height $z_t$|
|$q_{0}$| Mixing ratio at the surface|

The bulk coefficients ($C_H$ & $C_E$) are independent of the wind speed and only depend on the stability of the atmosphere and the roughness of the surface, $z_{0_v}$. For snow the roughness length is a function of time, which increases linearly from that of fresh snow to that of firn. For ice the roughness length is constant.


<div class="alert alert-warning">
    <details><summary><b>
        With your knowledge from the previous notebooks, can you design an experiment to investigate the dependence of the roughness length on the calculations of the turbulent fluxes?
        </b> <i>Click me for a hint!</i></summary>
        Begin by taking a look at the options (edu_utils.print_options()). Then use the appropriate variables for a sensitivity study, much like in the previous  notebooks.
</div>

In [ ]:
# Write your code here.


In [ ]:
# Write your code here.


In [ ]:
# Write your code here. You can add more cells if needed.


#### Ground heat flux

### Sublimation



## Next steps
[Back to overview](welcome.ipynb)
